In [26]:
# Импорт необходимых библиотек из Flask
from flask import Flask # Фреймворк Flask для веб-приложений
from flask import render_template # Функция для рендеринга HTML-шаблонов
# Импорт библиотек для обработки данных и визуализации
import json # Библиотека для работы с JSON-данными
import pandas as pd # Библиотека для манипуляции и анализа данных
#Импорт Plotly для создания интерактивных графиков
import plotly # Основная библиотека Plotly
import plotly.express as px # Высокоуровневый интерфейс для визуализаций Plotly
import plotly.graph_objects as go # Низкоуровневый интерфейс для визуализаций Plotly
#Импорт клиента InfluxDB для взаимодействия с базой данных InfluxDB
import influxdb_client # Клиентская библиотека для InfluxDB
#Определение констант для подключения к InfluxDB
# Параметры для подключения к InfluxDB
from influxdb_client import InfluxDBClient, Point, WriteOptions 
from influxdb_client import InfluxDBClient

# Параметры для подключения к InfluxDB
bucket = "ebu"
org = "AKULA1"
token = "XDBtANrCelRK0vXOYd0yTkMDF2YPZsM_i2Y88ZUmSWIZXMTkoDRtYCMgfAb3D6icLsVegYVeVCwhNYGmnQdQvg=="
url = "http://localhost:8086"


# Создаем клиент для подключения к InfluxDB
client = InfluxDBClient(url=url, token=token, org=org)

# Инициализация приложения Flask
app = Flask(__name__)

# Определение маршрута для главной страницы
@app.route("/")
def render_results():
    try:
        # Получение имен колонок и данных метрик
        columns_names, data_metrics = metrics_data()

        # Генерация JSON для столбчатого графика
        graph_metrics_JSON = bar_graph_JSON()

        # Генерация JSON для линейного графика
        graph_prediction_JSON = line_graph_JSON()

        # Рендеринг шаблона index.html с сгенерированными данными
        return render_template('index.html',
                               graph_prediction_JSON=graph_prediction_JSON,
                               graph_metrics_JSON=graph_metrics_JSON,
                               data_metrics=data_metrics,
                               columns_names=columns_names)
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Функция для генерации JSON для линейного графика
def line_graph_JSON():
    try:
        # Чтение запроса Flux из файла
        query = open('reqwest.flux').read()
        # Выполнение запроса и получение таблиц
        tables = query_api.query(query, org=org)
        # Инициализация списков для хранения имен колонок и значений индексов
        columns = []
        for table in tables:
            # Извлечение первого имени поля из записей
            columns.append(table.records[0].values.get("_field"))

        indexes = []  # Список для хранения значений индексов (временных меток)
        for table in tables:
            for record in table.records:
                indexes.append(record.get_time())  # Получение временной метки
            break  # Выход после обработки первой таблицы

        # Создание DataFrame для хранения данных из InfluxDB
        infl_data_df = pd.DataFrame(columns=columns, index=indexes)
        # Заполнение DataFrame данными из записей
        for table in tables:
            data = []
            for record in table.records:
                data.append(record.get_value())  # Получение значения для каждой записи

            # Присвоение данных соответствующему столбцу в DataFrame
            infl_data_df[record.values.get("_field")] = data

        # Закрытие соединения с клиентом InfluxDB
        client.close()
        # Назначение имени индекса DataFrame
        infl_data_df.index.name = 'time'
        # Создание линейного графика с использованием Plotly
        fig_line_plot = px.line(infl_data_df)
        # Обновление макета линейного графика
        fig_line_plot.update_layout(
            autosize=False,  # Отключение автоматического изменения размера
            width=900,  # Установка ширины графика
            height=500  # Установка высоты графика
        )
        # Преобразование графика в формат JSON для рендеринга
        graph_prediction_JSON = json.dumps(fig_line_plot, cls=plotly.utils.PlotlyJSONEncoder)

        # Возвращение JSON-представления линейного графика
        return graph_prediction_JSON
    except Exception as e:
        raise RuntimeError(f"Error generating line graph JSON: {str(e)}")

# Функция для генерации JSON для столбчатого графика
def bar_graph_JSON():
    try:
        # Получение данных метрик
        _, data_metrics = metrics_data()

        # Проверка наличия столбца 'R2'
        if 'R2' not in data_metrics.columns:
            raise KeyError("Столбец 'R2' отсутствует в данных метрик.")

        # Создание столбчатого графика для MAE (средней абсолютной ошибки)
        fig_bar_plot = px.bar(data_metrics, y='MAE')

        # Добавление вторичного графика для значений R2
        fig_bar_plot.add_trace(go.Bar(x=data_metrics.index, y=data_metrics['R2'],
                                      name='Secondary Value', yaxis='y2'))

        # Обновление макета столбчатого графика
        fig_bar_plot.update_layout(
            autosize=False,  # Отключение автоматического изменения размера
            width=500,  # Установка ширины графика
            height=500,  # Установка высоты графика
            yaxis=dict(title='MAE'),  # Заголовок для основного y-оси
            yaxis2=dict(title='R2', overlaying='y', side='right'),  # Заголовок для вторичной у-оси
            barmode='group'  # Группировка столбцов
        )

        # Преобразование графика в формат JSON для рендеринга
        graph_metrics_JSON = json.dumps(fig_bar_plot, cls=plotly.utils.PlotlyJSONEncoder)

        # Возвращение JSON-представления столбчатого графика
        return graph_metrics_JSON
    except Exception as e:
        raise RuntimeError(f"Error generating bar graph JSON: {str(e)}")

# Функция для получения данных метрик из CSV-файла
def metrics_data():
    try:
        # Чтение CSV-файла в DataFrame
        data_metrics = pd.read_csv('data.csv', header=0, index_col='Model')
        # Извлечение имен колонок из DataFrame
        columns_names = data_metrics.columns
        # Преобразование всех данных в DataFrame в тип float
        data_metrics = data_metrics.astype(float)
        # Возвращение имен колонок и данных метрик
        return columns_names, data_metrics
    except Exception as e:
        raise RuntimeError(f"Error reading metrics data: {str(e)}")

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\AKULA\anaconda3\envs\inf\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [33]:
import pandas as pd
import influxdb_client
client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)
query_api = client.query_api()
# Составляем запрос на получение из БД
# значений сенсора с ID = 1, по всем
# имеющимся временным меткам
query = """from(bucket: "ebu")
 |> range(start: 0 )
 |> filter(fn: (r) => r["_measurement"] == "sensors_data")
 |> filter(fn: (r) => r["sensorid"] == "10")
  """
tables = query_api.query(query, org="AKULA1")
# print(tables)
# При помощи цикла запишем возвращаемые строки
# в лист data
data = []
for table in tables:
  for record in table.records:
    # print(record)
    data.append({'timestamp': record.get_time(),
                 'value': record.get_value()})
client.close()
# Преобразуем лист в датафрейм
df = pd.DataFrame(data)
# На основе временных меток сформируем
# временные признаки
df['minute'] = df['timestamp'].dt.minute.astype('int64')
df['hour'] = df['timestamp'].dt.hour.astype('int64')
df['second'] = df['timestamp'].dt.second.astype('int64')
# На основе целевой колонки сформируем
# дополнитлеьный сдвиговый признак
df['shift_5'] = df['value'].shift(5)
df.index = pd.DatetimeIndex(df['timestamp'])
df = df.drop(['timestamp'], axis=1)
df.index.sort_values()
# df['value'].plot()

df = df.dropna()

In [34]:
df

,value,minute,hour,second,shift_5
timestamp,,,,,
2024-04-18 00:00:05+00:00,80.298698,0,0,5,21.799012
2024-04-18 00:00:06+00:00,66.404557,0,0,6,16.546189
2024-04-18 00:00:07+00:00,19.838376,0,0,7,3.998126
2024-04-18 00:00:08+00:00,40.502318,0,0,8,49.105791
2024-04-18 00:00:09+00:00,29.348776,0,0,9,42.702048
...,...,...,...,...,...
2024-04-18 00:01:35+00:00,76.953710,1,0,35,98.045212
2024-04-18 00:01:36+00:00,90.675148,1,0,36,70.220119
2024-04-18 00:01:37+00:00,1.172922,1,0,37,86.956022


In [50]:
# Параметры для подключения к InfluxDB
bucket = "ebu"
org = "AKULA1"
token = "XDBtANrCelRK0vXOYd0yTkMDF2YPZsM_i2Y88ZUmSWIZXMTkoDRtYCMgfAb3D6icLsVegYVeVCwhNYGmnQdQvg=="
url = "http://localhost:8086"

# Создание экземпляра клиента InfluxDB
client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

# Создание экземпляра API запросов для выполнения запросов
query_api = client.query_api()


In [55]:

# Функция для получения данных метрик из InfluxDB
def metrics_data():
    try:
        # Чтение запроса Flux из файла
        query = open('reqwest.flux').read()
        # Выполнение запроса и получение таблиц
        tables = query_api.query(query, org=org)
        # Инициализация списков для хранения имен колонок и значений индексов
        columns = []
        for table in tables:
            # Извлечение первого имени поля из записей
            columns.append(table.records[0].values.get("_field"))

        indexes = []  # Список для хранения значений индексов (временных меток)
        for table in tables:
            for record in table.records:
                indexes.append(record.get_time())  # Получение временной метки
            break  # Выход после обработки первой таблицы

        # Создание DataFrame для хранения данных из InfluxDB
        infl_data_df = pd.DataFrame(columns=columns, index=indexes)
        # Заполнение DataFrame данными из записей
        for table in tables:
            data = []
            for record in table.records:
                data.append(record.get_value())  # Получение значения для каждой записи

            # Присвоение данных соответствующему столбцу в DataFrame
            infl_data_df[record.values.get("_field")] = data

        # Закрытие соединения с клиентом InfluxDB
        client.close()
        # Назначение имени индекса DataFrame
        infl_data_df.index.name = 'time'
        # Преобразование DataFrame в формат, пригодный для метрик
        infl_data_df = infl_data_df.T  # Транспонирование DataFrame
        infl_data_df.columns = columns  # Установка имен колонок
        # Возвращение имен колонок и данных метрик
        return columns, infl_data_df
    except Exception as e:
        raise RuntimeError(f"Error reading metrics data: {str(e)}")

In [56]:
metrics_data()

([],
 Empty DataFrame
 Columns: []
 Index: [])

In [84]:
from influxdb_client import InfluxDBClient

# Параметры для подключения к InfluxDB
bucket = "ebu"
org = "AKULA1"
token = "XDBtANrCelRK0vXOYd0yTkMDF2YPZsM_i2Y88ZUmSWIZXMTkoDRtYCMgfAb3D6icLsVegYVeVCwhNYGmnQdQvg=="
url = "http://localhost:8086"

# Создание экземпляра клиента InfluxDB
client = InfluxDBClient(
    url=url,
    token=token,
    org=org
)

# Создание экземпляра API запросов для выполнения запросов
query_api = client.query_api()

# Чтение запроса Flux из файла
with open('reqwest.flux', 'r') as file:
    query = file.read()
    print(query)
# Выполнение запроса и получение таблиц
tables = query_api.query(query, org=org)
# print(tables)
# Вывод данных для проверки
for table in tables:
    for record in table.records:
        print(record)


from(bucket: "ebu")
  |> range(start: 2024-04-18T00:00:00Z, stop: 2024-04-18T00:02:00Z)
  |> filter(fn: (r) => r["_measurement"] == "sensors_data_prediction")
  |> filter(fn: (r) => r["SensorID"] == "10")
  |> yield(name: "mean")

FluxRecord() table: 0, {'result': 'mean', 'table': 0, '_start': datetime.datetime(2024, 4, 18, 0, 0, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 4, 18, 0, 2, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2024, 4, 18, 0, 1, 35, tzinfo=datetime.timezone.utc), '_value': 40.759470702346675, 'DataType': 'prediction', 'SensorID': '10', '_field': 'sgd', '_measurement': 'sensors_data_prediction', 'mae': '36.77126796194519', 'r2': '-0.16506433597301462'}
FluxRecord() table: 0, {'result': 'mean', 'table': 0, '_start': datetime.datetime(2024, 4, 18, 0, 0, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 4, 18, 0, 2, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2024, 4, 18, 0, 1, 36, tzinfo=datetime.timezone.utc

In [98]:
import pandas as pd
from influxdb_client import InfluxDBClient

# Параметры для подключения к InfluxDB
bucket = "eee"
org = "AKULA1"
token = "XDBtANrCelRK0vXOYd0yTkMDF2YPZsM_i2Y88ZUmSWIZXMTkoDRtYCMgfAb3D6icLsVegYVeVCwhNYGmnQdQvg=="
url = "http://localhost:8086"

# Создание экземпляра клиента InfluxDB
client = InfluxDBClient(
    url=url,
    token=token,
    org=org
)

# Создание экземпляра API запросов для выполнения запросов
query_api = client.query_api()

# Функция для получения данных метрик из InfluxDB
def metrics_data():
    try:
        # Чтение запроса Flux из файла
        with open('reqwest.flux', 'r') as file:
            query = file.read()
        # Выполнение запроса и получение таблиц
        tables = query_api.query(query, org=org)
        # Инициализация списков для хранения имен колонок и значений индексов
        columns = []
        for table in tables:
            for record in table.records:
                field = record.get_field()
                if field not in columns:
                    columns.append(field)

        indexes = []  # Список для хранения значений индексов (временных меток)
        for table in tables:
            for record in table.records:
                indexes.append(record.get_time())  # Получение временной метки
            break  # Выход после обработки первой таблицы

        # Создание DataFrame для хранения данных из InfluxDB
        infl_data_df = pd.DataFrame(columns=columns, index=indexes)
        # Заполнение DataFrame данными из записей
        for table in tables:
            for record in table.records:
                field = record.get_field()
                value = record.get_value()
                infl_data_df.at[record.get_time(), field] = value

        # Закрытие соединения с клиентом InfluxDB
        client.close()
        # Назначение имени индекса DataFrame
        infl_data_df.index.name = 'time'
        # Возвращение имен колонок и данных метрик
        return columns, infl_data_df
    except Exception as e:
        raise RuntimeError(f"Error reading metrics data: {str(e)}")

# Вызов функции для проверки
columns_names, data_metrics = metrics_data()
print("Columns:", columns_names)
print("Data Metrics:", data_metrics)


RuntimeError: Error reading metrics data: '_field'

In [ ]:
import pandas as pd
from influxdb_client import InfluxDBClient

# Параметры для подключения к InfluxDB
bucket = "eee"
org = "AKULA1"
token = "XDBtANrCelRK0vXOYd0yTkMDF2YPZsM_i2Y88ZUmSWIZXMTkoDRtYCMgfAb3D6icLsVegYVeVCwhNYGmnQdQvg=="
url = "http://localhost:8086"

# Создание экземпляра клиента InfluxDB
client = InfluxDBClient(
    url=url,
    token=token,
    org=org
)

# Создание экземпляра API запросов для выполнения запросов
query_api = client.query_api()

# Функция для получения данных метрик из InfluxDB
def metrics_data():
    try:
        # Flux запрос для извлечения данных
        query = """
        from(bucket: "eee")
          |> range(start: 2024-04-18T00:00:00Z, stop: 2024-04-18T00:02:00Z)
          |> filter(fn: (r) => r["_measurement"] == "sensors_data_prediction")
          |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
        """
        print("Flux Query:", query)  # Отладочное сообщение

        # Выполнение запроса и получение таблиц
        tables = query_api.query(query, org=org)
        print("Tables received:", tables)  # Отладочное сообщение

        # Инициализация списков для хранения имен колонок и значений индексов
        columns = set()
        for table in tables:
            for record in table.records:
                for key in record.values.keys():
                    if key not in ["_start", "_stop", "_time", "result", "table"]:
                        columns.add(key)

        indexes = []  # Список для хранения значений индексов (временных меток)
        for table in tables:
            for record in table.records:
                indexes.append(record.get_time())  # Получение временной метки
            break  # Выход после обработки первой таблицы

        # Создание DataFrame для хранения данных из InfluxDB
        infl_data_df = pd.DataFrame(columns=list(columns), index=pd.to_datetime(indexes))
        # Заполнение DataFrame данными из записей
        for table in tables:
            for record in table.records:
                for key in record.values.keys():
                    if key not in ["_start", "_stop", "_time", "result", "table"]:
                        infl_data_df.at[record.get_time(), key] = record.values[key]

        # Закрытие соединения с клиентом InfluxDB
        client.close()
        # Назначение имени индекса DataFrame
        infl_data_df.index.name = 'time'
        # Возвращение имен колонок и данных метрик
        return list(columns), infl_data_df
    except Exception as e:
        raise RuntimeError(f"Error reading metrics data: {str(e)}")

# Вызов функции для проверки
columns_names, data_metrics = metrics_data()
print("Columns:", columns_names)
print("Data Metrics:", data_metrics)


Flux Query: 
        from(bucket: "eee")
          |> range(start: 2024-04-18T00:00:00Z, stop: 2024-04-18T00:02:00Z)
          |> filter(fn: (r) => r["_measurement"] == "sensors_data_prediction")
          |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
        
Tables received: [<FluxTable: 11 columns, 5 records>]
Columns: ['mae', '_measurement', 'sgd', 'DataType', 'SensorID', 'r2']
Data Metrics:                                           mae             _measurement  \
time                                                                     
2024-04-18 00:01:35+00:00  36.805871293260054  sensors_data_prediction   
2024-04-18 00:01:36+00:00  36.805871293260054  sensors_data_prediction   
2024-04-18 00:01:37+00:00  36.805871293260054  sensors_data_prediction   
2024-04-18 00:01:38+00:00  36.805871293260054  sensors_data_prediction   
2024-04-18 00:01:39+00:00  36.805871293260054  sensors_data_prediction   

                                 sgd    DataType Sensor